In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]


In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-13 01:50:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.04MB/s    in 0.2s    

2020-11-13 01:50:51 (5.04 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [13]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://jake-data-bootcamp-bucket.s3.amazonaws.com/US_Accidents_June20.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("US_Accidents_June20.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+----+--------+-----+--------+-------------------+-------------------+---------+----------+-------+-------+------------+--------------------+------+--------------------+----+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID|  Source|  TMC|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|Number|              Street|Side|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Conditi

## Count to make sure all data is there.


In [14]:
print(df.count())

3513617


## Examine the schema

In [15]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- TMC: double (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: string (nullable = true)
 |-- End_Time: string (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable =

## Rename columns

In [19]:
df1 = df.withColumnRenamed("State", "state_") \
        .withColumnRenamed("ID", "id_") \
        .withColumnRenamed("Source", "source_") \
        .withColumnRenamed("Number", "number_") \
        .withColumnRenamed("Distance(mi)", "distance_mi") \
        .withColumnRenamed("Temperature(F)", "temperature_f") \
        .withColumnRenamed("Visibility(mi)", "visibility_mi") \
        .withColumnRenamed("Wind_Speed(mph)", "wind_speed_mph") \
        .withColumnRenamed("Precipitation(in)", "precipitation_in")
df1.show(5)

+---+--------+-----+--------+-------------------+-------------------+---------+----------+-------+-------+-----------+--------------------+-------+--------------------+----+------------+----------+------+----------+-------+----------+------------+-------------------+-------------+-------------+-----------+------------+-------------+--------------+--------------+----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|id_| source_|  TMC|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|End_Lat|End_Lng|distance_mi|         Description|number_|              Street|Side|        City|    County|state_|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|temperature_f|Wind_Chill(F)|Humidity(%)|Pressure(in)|visibility_mi|Wind_Direction|wind_speed_mph|precipitation_in|Weather_Condition|Ameni

## Create a new DataFrame for crashes

In [21]:
df2 = df1.select(["id_", "Severity", "Start_Time", "End_Time", 
                  "Start_Lat", "Start_Lng", "Description", "Street", 
                  "City", "County", "state_", "Zipcode", "Country", 
                  "Timezone", "temperature_f", "visibility_mi", 
                  "wind_speed_mph", "precipitation_in", "Weather_Condition",
                  "Traffic_Signal", "Sunrise_Sunset"])
df2.show(5)

+---+--------+-------------------+-------------------+---------+----------+--------------------+--------------------+------------+----------+------+----------+-------+----------+-------------+-------------+--------------+----------------+-----------------+--------------+--------------+
|id_|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|         Description|              Street|        City|    County|state_|   Zipcode|Country|  Timezone|temperature_f|visibility_mi|wind_speed_mph|precipitation_in|Weather_Condition|Traffic_Signal|Sunrise_Sunset|
+---+--------+-------------------+-------------------+---------+----------+--------------------+--------------------+------------+----------+------+----------+-------+----------+-------------+-------------+--------------+----------------+-----------------+--------------+--------------+
|A-1|       3|2016-02-08 05:46:00|2016-02-08 11:00:00|39.865147|-84.058723|Right lane blocke...|              I-70 E|      Dayton|Montgomer

## Write DataFrame to RDS

In [22]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://db-inst-on-aws.cfx0x80oubi1.us-east-1.rds.amazonaws.com:5432/db_on_aws"
config = {"user":"root",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [24]:
# Write DataFrame to table

df2.write.jdbc(url=jdbc_url, table='crashes', mode=mode, properties=config)